# Prepare

In [1]:
! pip install pyexcel pyexcel-xlsx

     |████████████████████████████████| 88 kB 4.7 MB/s 
     |████████████████████████████████| 44 kB 2.0 MB/s 
     |████████████████████████████████| 242 kB 41.4 MB/s 
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9


In [2]:
# create multiple xlsx files

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.28
"""

# import
import time
import random
import os
import pyexcel as px
import shutil

#
print("Process Start.")
start_time = time.time()

#
NUM_SAMPLES = 1000

#
alphabet_samples = "abcdefghizklmnopqrstuvwxyz1234567890"

first_name_samples = "김이박최정강조윤장임"
middle_name_samples = "민서예지도하주윤채현지"
last_name_samples = "준윤우원호후서연아은진"

#
def random_string(length):
    result = ""

    for i in range(length):
        result += random.choice(alphabet_samples)

    return result


def random_name():
    result = ""

    result += random.choice(first_name_samples)
    result += random.choice(middle_name_samples)
    result += random.choice(last_name_samples)

    return result

#
if os.path.exists("personal_info"):
    shutil.rmtree("personal_info")
os.mkdir("personal_info")

#
HEADER = ["name", "age", "e-mail", "division", "telephone", "sex"]

#
for i in range(NUM_SAMPLES):

    #
    name = random_name()

    #
    filename = "personal_info/" + str(i) + "_" + name + ".xlsx"

    #
    contents = []

    contents.append(name)
    contents.append(str(time.time())[-2:])
    contents.append(random_string(8) + "@bhban.com")
    contents.append(random_string(3))
    contents.append("010-" + str(time.time())[-4:] + "-" + str(time.time())[-6:-2])
    contents.append(random.choice(["male", "female"]))

    #
    RESULT = [HEADER, contents]

    #
    px.save_as(array=RESULT, dest_file_name=filename)

#
print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Process Start.
Process Done.
Peter's code 8.413665056228638 seconds.


In [3]:
# create merged_ID.xlsx

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.27
"""

# import
import time
import os
import pyexcel as px
import sys

#
print("Process Start.")
start_time = time.time()

"""
# original
directory = sys.argv[1]
"""

# revised
directory = "personal_info/"

#
outfile_name = "merged_ID.xlsx"

input_files = os.listdir(directory)

#
CONTENTS = []

#
for filename in input_files:
    
    # exception
    if ".xlsx" not in filename:
        continue

    #
    data_array = px.get_array(file_name=directory + "/" + filename)

    # header = data_array[0]
    # data_array = data_array[1:]

    header = data_array.pop(0)
    
    if len(CONTENTS) == 0:
        CONTENTS.append(header)

    CONTENTS += data_array

#
px.save_as(array=CONTENTS, dest_file_name=outfile_name)

#
print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Process Start.
Process Done.
Peter's code 7.281777620315552 seconds.


In [4]:
# destroy files

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.28
"""

# import
import time
import os
import pyexcel as px
import sys
import random

#
print("Process Start.")
start_time = time.time()

"""
# original

directory = sys.argv[1]
percent = float(sys.argv[2]) / 100
"""

# revised
directory = "personal_info"
percent = 30 / 100

#
input_files = os.listdir(directory)

DESTROY_COUNT = int(len(input_files) * percent)

#
random.shuffle(input_files)

#
destroy_them = input_files[:DESTROY_COUNT]

#
shift_1 = DESTROY_COUNT/3*2
shift_2 = DESTROY_COUNT/3

#
for filename in destroy_them:

    # exception
    if ".xlsx" not in filename:
        continue

    #
    file = px.get_sheet(file_name=directory + "/" + filename)

    os.remove(directory + "/" + filename)

    #
    data_array = file.array

    num_columns = len(data_array[0])
    victim = random.randint(0, num_columns - 1)

    # 1st Disaster
    if DESTROY_COUNT > shift_1:
        file.delete_columns([victim])

    # 2nd Disater
    elif DESTROY_COUNT > shift_2:
        file.column += file.column[victim].copy()
    
    # 3rd Disaster
    else:
        CAT = ["Peter's Kitten" for i in range(file.number_of_rows())]
        file.column[victim] = CAT

    #
    px.save_as(array=data_array, dest_file_name=directory + "/" + filename)

    #
    DESTROY_COUNT -= 1

#
print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Process Start.
Process Done.
Peter's code 4.83501410484314 seconds.


# RPA design code
- 목표 정하기
  1. 파일을 읽어와 헤더를 비교하기
  1. 헤더가 템플릿과 같으면 내용물만 뽑아오기
- 목표를 달성하는 데 필요한 작업 쪼개기
  1. 엑셀 파일을 하나씩 읽어오기
  1. 헤더 추출하기
  1. 여러 개의 엑셀 파일을 한 개로 합치기

#### 1. 엑셀 파일을 하나씩 읽어오기
- ```xls_file = px.get_array(filename)```

#### 2. 헤더 추출하기
- ```header = xls_file[0]```

#### 3. 여러 개의 엑셀 파일을 한 개로 합치기
- ```px.save_as()```

In [8]:
%%writefile merge_correct_xlsx.py
#!usr/bin/bash python
#-*-coding:utf-8

import time
import os
import pyexcel as px
import sys

#
print("Process Start.")
start_time = time.time()

#
template = sys.argv[1]
directory = sys.argv[2]

#
input_files = os.listdir(directory)
HEADER = px.get_array(file_name=template)[0]
CONTENTS = [HEADER]

#
for filename in input_files:
    # exception
    if ".xlsx" not in filename:
        continue
    
    file = px.get_array(file_name=directory + "/" + filename)

    header = file[0]

    if HEADER != header:
        continue
    
    CONTENTS += file[1:]

#
px.save_as(array=CONTENTS, dest_file_name="merged_FILE.xlsx")

print("Total " + str(len(CONTENTS) - 1) + " files were merged.")
print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Overwriting merge_correct_xlsx.py


##### README.md
- ```%run merged_correct_xlsx.py <TEMPLATE> <DIRECTORY>```

In [9]:
%run merge_correct_xlsx.py merged_ID.xlsx personal_info

Process Start.
Total 700 files were merged.
Process Done.
Peter's code 7.5708582401275635 seconds.


In [12]:
##### 결과 확인
import pandas as pd
display(pd.read_excel(io="merged_FILE.xlsx").tail())

,name,age,e-mail,division,telephone,sex
695,박서연,58,0q903ysr@bhban.com,y2q,010-8177-1381,female
696,이윤호,55,kvgns438@bhban.com,rly,010-4169-7417,male
697,강현아,88,aw8z00vs@bhban.com,z3z,010-8005-1580,female
698,김윤준,14,n7hfqx72@bhban.com,tpc,010-9128-4913,female
699,최예아,57,ae56um2t@bhban.com,zt6,010-0928-8092,female


## Summary
- ```CONTENTS += file[1:]``` 과 같이 범용적인 코드를 만드는 것이 핵심이다.